In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict
from settings import *
from math import sqrt

# import xgboost as xgb
from sklearn import preprocessing, neighbors, svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, \
  RandomForestRegressor, AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

DIR_DATA = "data"
DIR_PROCESSED = "processed"
LABEL = "rating"
NON_PREDICTORS = [LABEL, "int_rating"]#, "name","anime_id"]
CV_FOLDS = 5

In [2]:
ratings = pd.read_csv(DIR_DATA + '/rating.csv')

In [3]:
# animeRatings = ratings.pivot_table(index=['user_id'], columns='anime_id', values='rating')
# animeRatings = animeRatings.replace(-1, np.nan)
# a_file = open(DIR_PROCESSED + '/ratings_pivot_table.pickle', 'wb')
# pickle.dump(animeRatings.to_sparse(), a_file)
# a_file.close()

# corrMatrix = animeRatings.corr()
# a_file = open(DIR_PROCESSED + '/ratings_corr_pivot_table.pickle', 'wb')
# pickle.dump(corrMatrix, a_file)
# a_file.close()

a_file = open(DIR_PROCESSED + '/ratings_pivot_table.pickle', 'rb')
anime_ratings = pickle.load( a_file)
a_file.close()

a_file = open(DIR_PROCESSED + '/ratings_corr_pivot_table.pickle', 'rb')
corrMatrix = pickle.load( a_file)
a_file.close()


In [ ]:
anime_ratings.head()

In [ ]:
corrMatrix.head()


In [3]:
# from scipy.sparse import csr_matrix
# animeMatrix = animeRatings.replace(np.nan, 0)


In [3]:
ratings["rating"] = ratings.rating.replace(-1, np.nan)
userRatings = ratings.pivot_table(index=['anime_id'], columns='user_id', values='rating')
userRatings.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
anime_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,10.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,10.0,NaN
6,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
userCorrMatrix = userRatings.corr(method='pearson', min_periods=30)
userCorrMatrix.head()

MemoryError: 

In [5]:
a_file = open(DIR_PROCESSED + '/user_anime_id_pivot_table.pickle', 'wb')
pickle.dump(userRatings.to, a_file)
a_file.close()

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [ ]:
sparseUserRatings = userRatings.to_sparse()

In [12]:
all_ratings = ratings.rating[ratings.rating >=0].values

np.mean(all_ratings)

7.8084969468574732

In [5]:
np.std(all_ratings)

1.0267037518754754

In [6]:
MAE = np.abs(all_ratings - np.mean(all_ratings))
MAE = sum(MAE) / len(MAE)
print('Expected MAE ', MAE)


Expected MAE  0.80050624386


In [7]:
MSE = (all_ratings - np.mean(all_ratings))**2
MSE = sum(MSE) / len(MSE)
print('Expected MSE', MSE)

Expected MSE 1.05412059412


In [8]:
RMSE = np.sqrt(MSE)
print('Expected RMSE', RMSE)

Expected RMSE 1.02670375188


In [4]:
a_file = open(DIR_PROCESSED + '/one_hot_encoded_anime.pickle', 'rb')
anime = pickle.load(a_file)
a_file.close()

all_ratings = anime.rating[anime.rating >=0].values
np.mean(all_ratings)

6.4739016909814318